In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("WANDB_API_KEY") 

In [ ]:
import wandb
wandb.login(key=my_secret)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm.auto import tqdm
from time import time
import warnings
from sklearn.impute import KNNImputer
warnings.filterwarnings("ignore")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

# Parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 71 # for reproducibility
SATELLITE_EXTRA_LAYER = False # for adding layer to pre-built models
STREET_EXTRA_LAYER = False # for adding layer to pre-built models
BATCH_SIZE = 400 # number of samples processed in one training batch
EPOCHS = 40 # mber of training iterations over the entire dataset
LEARNING_RATE = 1e-2 # to update model weights
IMG_SIZE = 224 # input image
NUM_FROZEN_LAYERS = 0 # 0 indicates that all layers are trainable
DROPOUT = 0.5
LABELS = ['AQI','PM2.5', 'PM10','O3', 'CO', 'SO2', 'NO2']

SATELLITE_ENCODER = "mobilenet_v3_small" #'vit_base_32'
STREET_ENCODER = "mobilenet_v3_small"#'vit_base_32'
ATTENTION = "sigmoid_gated" #"softmax_gated" # , "sigmoid_gated" ,  "cross" 
RUN_NAME = f'st-{SATELLITE_ENCODER}_sv-{STREET_ENCODER}_attn-{ATTENTION}'
TAGS = [f'Satellite_{SATELLITE_ENCODER}_EL-{SATELLITE_EXTRA_LAYER}', f'Street_{STREET_ENCODER}_EL-{STREET_EXTRA_LAYER}', '7Bands', ATTENTION]

print(f"Using device: {device}")

In [ ]:
run = wandb.init(
    project="AQI&PollutantsDetection",
    name=RUN_NAME,
    tags=TAGS,
    resume='allow',
    allow_val_change=True,
    config={
        "SEED": SEED,
        "SATELLITE_EXTRA_LAYER": SATELLITE_EXTRA_LAYER,
        "STREET_EXTRA_LAYER": STREET_EXTRA_LAYER,
        "BATCH_SIZE": BATCH_SIZE,
        "EPOCHS": EPOCHS,
        "LEARNING_RATE": LEARNING_RATE,
        "IMG_SIZE": IMG_SIZE,
        "NUM_FROZEN_LAYERS": NUM_FROZEN_LAYERS,
        "DROPOUT": DROPOUT,
        "DEVICE": str(device),
        "SATELLITE_ENCODER": SATELLITE_ENCODER,
        "STREET_ENCODER": STREET_ENCODER,
        "LABELS": LABELS
    }
)


# Dataset
- Loads dataset: https://www.kaggle.com/datasets/adarshrouniyar/air-pollution-image-dataset-from-india-and-nepal
- CSV consists of places and AQI measures
- We have added an additional field for sattelite image path which was collected separately using Google earth engine.
- Fields: Location, Filename (the street view image), Normalized_Filename (satellite image), Year, Month, Day, Hour, AQI, PM2.5, PM10, O3, CO, SO2, NO2, and AQI_Class

In [ ]:
dataset_path = '/kaggle/input/aqi-dataset/dataset'
# dataset_path = './dataset'
train_csv = pd.read_csv(os.path.join(dataset_path, 'train_data.csv'))
val_csv = pd.read_csv(os.path.join(dataset_path, 'val_data.csv'))
test_csv = pd.read_csv(os.path.join(dataset_path, 'test_data.csv'))

# train_csv.interpolate(method='linear', inplace=True)
# val_csv.interpolate(method='linear', inplace=True)
# test_csv.interpolate(method='linear', inplace=True)

In [ ]:
combined_df = pd.concat([train_csv, val_csv, test_csv])

# Perform interpolation and KNN imputation
# combined_df.interpolate(method='linear', inplace=True)
knn_imputer = KNNImputer(n_neighbors=5)
combined_df[['SO2', 'NO2', 'O3', 'CO']] = knn_imputer.fit_transform(combined_df[['SO2', 'NO2', 'O3', 'CO']])

# Split them back into separate datasets
train_csv = combined_df.iloc[:len(train_csv)]
val_csv = combined_df.iloc[len(train_csv):len(train_csv) + len(val_csv)]
test_csv = combined_df.iloc[len(train_csv) + len(val_csv):]

# Custom Dataset

#### **1. Initialization (`__init__`)**  
   - Takes a **CSV file** (*pandas DataFrame*) containing **filenames and labels**.  
   - **`satellite_img_dir`** & **`street_img_dir`**: Directories where **satellite** and **street images** are stored.  
   - **`label`**: Column name in the CSV corresponding to the **target label**.  
   - **`satellite_transform`** & **`street_transform`**: Optional transformations (e.g., **resizing, normalization**) for images.  

#### **2. Dataset Length (`__len__`)**  
   - Returns the **total number of samples** (rows) in the CSV file.  

#### **3. Get Item (`__getitem__`)**  
   - Retrieves the **street image** and **satellite image** paths from the CSV.  
   - Loads the images using the **`load_image()`** method.  
   - Applies the **respective transformations** if provided.  
   - Converts the **label** to a **PyTorch tensor** of type `float32`.  
   - Returns a tuple: **`(street_image, satellite_image, label)`**.  

#### **4. Image Loading (`load_image`)**  
   - Opens an image using **PIL** (`Image.open(file_path)`).  
   - Applies **transformations** if specified.  
   - Returns the **processed image**. 🚀  

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, satellite_img_dir, street_img_dir, label, satellite_transform=None, street_transform=None):
        self.data = csv_file
        self.street_img_dir = street_img_dir
        self.satellite_img_dir = satellite_img_dir
        self.label = label
        self.satellite_transform = satellite_transform
        self.street_transform = street_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        street_image_path = os.path.join(self.street_img_dir, self.data['Filename'].iloc[idx])
        satellite_image_path = os.path.join(self.satellite_img_dir, self.data['Normalized_Filename'].iloc[idx]+'.npy' )
        
        street_image = self.load_image(street_image_path, self.street_transform)
        # satellite_image = self.load_image(satellite_image_path, self.satellite_transform)
        satellite_image = self.load_npy(satellite_image_path, self.satellite_transform)
        labels = torch.tensor(self.data[self.label].iloc[idx], dtype=torch.float32)
        
        return street_image, satellite_image, labels
        
    
    def load_image(self, file_path, transform=None):
        img = Image.open(file_path)
        if transform:
            img = transform(img)
        return img
    
    def load_npy(self, file_path, transform=None):
        img = np.load(file_path)
        img = torch.tensor(img, dtype=torch.float32)

        # If the data is in [0, 255], scale it to [0, 1]
        if img.max() > 1.0:  
            img = img / 255.0  

        # Ensure shape (C, H, W) if it's (H, W, C)
        if img.ndimension() == 3 and img.shape[-1] == 7:  
            img = img.permute(2, 0, 1)  
        if transform:
            img = transform(img)
        return img
    

## Data normalization and tranformation

In [ ]:
image_net_means = [0.485, 0.456, 0.406] 
image_net_stds = [0.229, 0.224, 0.225]

satellite_net_means = [0.18168019890450375, 0.18805927958530722, 0.20592676343591497, 0.20806291225568016, 0.3423790143310607, 0.23654847637549638, 0.17482840221654344]

satellite_net_stds = [0.19048610465575523, 0.19615030016268702, 0.2125846014779801,  0.21476670175116374, 0.347457205638518, 0.2390436189214837, 0.17736793155031446]
# for data transformation
data_transforms = {
    'street_dev': transforms.Compose([
        transforms.ToTensor(),  # Converts (H, W, C) to (C, H, W)
        transforms.Normalize(tuple(image_net_means), tuple(image_net_stds))
    ]),
    'street_test': transforms.Compose([
        transforms.ToTensor(),  # Converts (H, W, C) to (C, H, W)
        transforms.Normalize(tuple(image_net_means), tuple(image_net_stds))
    ]),
    'satellite_dev': transforms.Compose([
        # transforms.ToTensor(),  # Converts (H, W, C) to (C, H, W)
        transforms.Resize((224, 224)),
        transforms.Normalize(tuple(satellite_net_means), tuple(satellite_net_stds))
    ]),
    'satellite_test': transforms.Compose([
        # transforms.ToTensor(),  # Converts (H, W, C) to (C, H, W)
        transforms.Resize((224, 224)),
        transforms.Normalize(tuple(satellite_net_means), tuple(satellite_net_stds))
    ])
}   
    
# image specific tranformations    
satellite_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomVerticalFlip(),
    transforms.Normalize(tuple(satellite_net_means), tuple(satellite_net_stds)),
])

street_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(), 
    transforms.Normalize(tuple(image_net_means), tuple(image_net_stds)),
])

## Dataloaders 

1. **Creating Datasets (`CustomDataset`)**  
   - `train_dataset`: Uses **augmented transforms** (`satellite_transform`, `street_transform`).  
   - `val_dataset` & `test_dataset`: Use **only normalization** (`data_transforms['dev']` & `data_transforms['test']`).  

2. **Data Loaders (`DataLoader`)**  
   - `train_loader`: Loads training data with **shuffling** for randomness.  
   - `val_loader` & `test_loader`: Load validation & test data **without shuffling** for consistency.  

Usage:
- Prepares **train, validation, and test sets** for deep learning models.  
- Uses **batch processing (`batch_size=BATCH_SIZE`)** for efficient training.  
- **`next(iter(train_loader))`** retrieves a single batch to inspect data. 🚀

In [ ]:
train_dataset = CustomDataset(train_csv, os.path.join(dataset_path, 'satellite_images'), os.path.join(dataset_path, 'All_img'), LABELS, satellite_transform, street_transform)

val_dataset = CustomDataset(val_csv, os.path.join(dataset_path, 'satellite_images'), os.path.join(dataset_path, 'All_img'), LABELS, data_transforms['satellite_dev'], data_transforms['street_dev'])

test_dataset = CustomDataset(test_csv, os.path.join(dataset_path, 'satellite_images'), os.path.join(dataset_path, 'All_img'), LABELS, data_transforms['satellite_test'], data_transforms['street_test'])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# data = next(iter(train_loader))

In [ ]:
len(train_loader), len(val_loader), len(test_loader)


In [ ]:
# temp = next(iter(train_loader))
# print(temp[0].shape, temp[1].shape, temp[2].shape)

# Loss Functions

In [ ]:
def loss_fn_regression(outputs, targets):
    """
    Loss function for regression
    """
    return nn.MSELoss()(outputs, targets)


def loss_fn_classification(outputs, targets):
    """
    Loss function for classification
    """

    return nn.CrossEntropyLoss()(outputs, targets)


def accuracy(outputs, targets):
    """
    Accuracy function
    """

    return (outputs.argmax(1) == targets).float().mean()


def rmse(outputs, targets):
    """
    RMSE function
    """

    return torch.sqrt(nn.MSELoss()(outputs, targets))


def mae(outputs, targets):
    """
    MAE function
    """

    return nn.L1Loss()(outputs, targets)

def multi_task_loss(aqi_pred, aqi_true, pm_pred, pm_true, gas_pred, gas_true, loss_fn=loss_fn_regression, weights=None):
    
    if weights is None:
        weights = [1.0, 1.0, 1.0]  # Default equal weights for AQI, PM, and Gases

    # print(weights)
    loss_aqi = loss_fn(aqi_pred, aqi_true)
    loss_pm = loss_fn(pm_pred, pm_true)
    loss_gas = loss_fn(gas_pred, gas_true)
    # print(loss_aqi, loss_pm, loss_gas)
    # Weighted sum of losses
    total_loss = weights[0] * loss_aqi + weights[1] * loss_pm + weights[2] * loss_gas
    return total_loss

def dynamic_task_weights(y_aqi, y_pm, y_gas):
    # Compute inverse variance (higher variance → lower weight)
    var_aqi = torch.var(y_aqi)
    var_pm = torch.var(y_pm)
    var_gas = torch.var(y_gas)

    weights = 1 / (torch.tensor([var_aqi, var_pm, var_gas]) + 1e-6)  # Avoid division by zero
    weights /= torch.sum(weights)  # Normalize weights to sum to 1
    return weights.tolist()

# Batch processing - forward pass
Processes a batch of data through the model and computes loss & metrics.  

1. **Inputs:**  
   - `data`: (street images, satellite images, targets)  
   - `model`: Deep learning model  
   - `device`: CPU/GPU  
   - `is_classification`: Flag for classification vs. regression  

2. **Steps:**  
   - Move data to **device**.  
   - Forward pass through **model**.  
   - Compute **loss** (classification or regression).  
   - Compute **accuracy** (only for classification).  
   - Compute **RMSE & MAE** (for both tasks).  

3. **Outputs:**  
   - Loss, Accuracy (0 for regression), RMSE, MAE 🚀

In [ ]:
def process_batch(data, model, device, is_classification):
    """Processes a batch and returns outputs, loss, and metrics."""
    street_img, satellite_img, targets = data
    street_img, satellite_img, targets = street_img.to(device), satellite_img.to(device), targets.to(device)
    
    outputs = model(street_img, satellite_img)
    
    if is_classification:
        loss = loss_fn_classification(outputs, targets)
        acc = accuracy(outputs, targets).item()
        rmse_score = 0
        mae_score = 0
    else:
        aqi_pred, pm_pred, gas_pred = outputs
        
        target_aqi = targets[:, 0].unsqueeze(1)  # AQI (1 value)
        target_pm = targets[:, 1:3]  # PM2.5, PM10 (2 values)
        target_gas = targets[:, 3:]  # O3, CO, SO2, NO2 (4 values)

        weights = dynamic_task_weights(target_aqi, target_pm, target_gas)
        # print(weights)
        
        loss = multi_task_loss(aqi_pred, target_aqi, pm_pred, target_pm, gas_pred, target_gas, weights=weights)
        acc = 0  # Not applicable for regression

        rmse_score = multi_task_loss(aqi_pred, target_aqi, pm_pred, target_pm, gas_pred, target_gas, rmse, weights=weights).item()
        mae_score = multi_task_loss(aqi_pred, target_aqi, pm_pred, target_pm, gas_pred, target_gas, mae, weights=weights).item()

    return loss, acc, rmse_score, mae_score
    

# Train and evaluation functions
1. **`train_fn(model, optimizer, scheduler, data_loader, device, is_classification=False)`**  
   - Trains the model for one epoch by iterating over the training dataset, computing loss, and updating weights using backpropagation.  
   - Tracks and prints batch-wise loss, RMSE, and MAE while applying learning rate scheduling.  

2. **`eval_fn(model, data_loader, device, is_classification=False)`**  
   - Evaluates the model on validation or test data without updating weights.  
   - Computes and returns the average loss, accuracy (if applicable), RMSE, and MAE for the entire dataset.  

3. **`train(model, optimizer, scheduler, train_loader, val_loader, test_loader, device, epochs=10, best_model_path='best_model.pth')`**  
   - Manages the full training process across multiple epochs, tracking performance on both training and validation sets.  
   - Saves the best-performing model based on validation loss and evaluates it on the test dataset after training. 🚀

In [ ]:
def train_fn(model, optimizer, scheduler, data_loader, device, is_classification=False):
    """Training function"""
    model.train()
    total_loss, total_acc, total_rmse, total_mae = 0, 0, 0, 0

    for i, data in tqdm(enumerate(data_loader), total=len(data_loader), desc="Training"):
        optimizer.zero_grad()

        loss, acc, rmse_score, mae_score = process_batch(data, model, device, is_classification)

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Aggregate metrics
        total_loss += loss.item()
        total_acc += acc
        total_rmse += rmse_score
        total_mae += mae_score
        
        wandb.log({
            "batch_loss": loss.item(),
            "batch_acc": acc,
            "batch_rmse": rmse_score,
            "batch_mae": mae_score
        })

        print(f"Batch: {i+1}/{len(data_loader)}, Loss: {total_loss / (i+1):.4f}, RMSE: {total_rmse / (i+1):.4f}, MAE: {total_mae / (i+1):.4f}", end='\r')

    print()  # Ensure proper formatting after tqdm

    epoch_loss = total_loss / len(data_loader)
    epoch_acc = total_acc / len(data_loader)
    epoch_rmse = total_rmse / len(data_loader)
    epoch_mae = total_mae / len(data_loader)

    # Log epoch-wise metrics to WandB
    wandb.log({
        "epoch_loss": epoch_loss,
        "epoch_acc": epoch_acc,
        "epoch_rmse": epoch_rmse,
        "epoch_mae": epoch_mae
    })

    return epoch_loss, epoch_acc, epoch_rmse, epoch_mae

In [ ]:
def eval_fn(model, data_loader, device, is_classification=False):
    """Evaluation function"""
    model.eval()
    total_loss, total_acc, total_rmse, total_mae = 0, 0, 0, 0

    with torch.no_grad():
        for i, data in tqdm(enumerate(data_loader), total=len(data_loader), desc="Evaluating"):
            loss, acc, rmse_score, mae_score = process_batch(data, model, device, is_classification)

            total_loss += loss.item()
            total_acc += acc
            total_rmse += rmse_score
            total_mae += mae_score

    val_loss = total_loss / len(data_loader)
    val_acc = total_acc / len(data_loader)
    val_rmse = total_rmse / len(data_loader)
    val_mae = total_mae / len(data_loader)

    # Log validation metrics to WandB
    wandb.log({
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_rmse": val_rmse,
        "val_mae": val_mae
    })

    return val_loss, val_acc, val_rmse, val_mae

In [ ]:
def train(model, optimizer, scheduler, train_loader, val_loader, test_loader, device, epochs=10, best_model_path='best_model.pth'):
    """Main training loop"""
    model.to(device)
    history = {"losses": [], "accuracies": [], "rmse_scores": [], "mae_scores": []}

    best_loss = np.inf
    # best_model_path = "best_model.pth"

    for epoch in tqdm(range(epochs), desc="Epochs"):
        train_metrics = train_fn(model, optimizer, scheduler, train_loader, device, is_classification=False)
        val_metrics = eval_fn(model, val_loader, device, is_classification=False)

        history["losses"].append((train_metrics[0], val_metrics[0]))
        history["accuracies"].append((train_metrics[1], val_metrics[1]))
        history["rmse_scores"].append((train_metrics[2], val_metrics[2]))
        history["mae_scores"].append((train_metrics[3], val_metrics[3]))

        print(f"Epoch: {epoch+1}/{epochs}, Train Loss: {train_metrics[0]:.4f}, Train Acc: {train_metrics[1]:.4f}, Train RMSE: {train_metrics[2]:.4f}, Train MAE: {train_metrics[3]:.4f}, Val Loss: {val_metrics[0]:.4f}, Val Acc: {val_metrics[1]:.4f}, Val RMSE: {val_metrics[2]:.4f}, Val MAE: {val_metrics[3]:.4f}")

        # Save best model
        if val_metrics[0] < best_loss:
            best_loss = val_metrics[0]
            torch.save(model.state_dict(), best_model_path)
            # wandb.save(best_model_path) 

    # Load best model and evaluate on test set
    model.load_state_dict(torch.load(best_model_path, weights_only=True))
    test_metrics = eval_fn(model, test_loader, device, is_classification=False)
    wandb.log({
        "test_loss": test_metrics[0],
        "test_acc": test_metrics[1],
        "test_rmse": test_metrics[2],
        "test_mae": test_metrics[3]
    })
    print(f"Test Loss: {test_metrics[0]:.4f}, Test Acc: {test_metrics[1]:.4f}, Test RMSE: {test_metrics[2]:.4f}, Test MAE: {test_metrics[3]:.4f}")

    return history

# Plotting metrics

In [ ]:
def plot_metrics(losses, accuracies, rmse_scores, mae_scores, model_name="Model Metrics"):
    """
    Plot training metrics with a title.
    """

    fig, ax = plt.subplots(2, 2, figsize=(20, 15))
    fig.suptitle(model_name, fontsize=20, fontweight="bold")  # Add model title

    ax[0, 0].plot(losses)
    ax[0, 0].set_title("Loss")
    ax[0, 0].legend(["Train", "Val"])

    ax[0, 1].plot(accuracies)
    ax[0, 1].set_title("Accuracy")
    ax[0, 1].legend(["Train", "Val"])

    ax[1, 0].plot(rmse_scores)
    ax[1, 0].set_title("RMSE")
    ax[1, 0].legend(["Train", "Val"])

    ax[1, 1].plot(mae_scores)
    ax[1, 1].set_title("MAE")
    ax[1, 1].legend(["Train", "Val"])

    plt.show()


# Base Model
1. Initializes base based on architecture selected.
2. Removes classification layer
3. Adjusts first layer based on number of channels
4. Adds final layers to extract features of image
5. Freeze layers depending on number of layers
6. Combine everything

In [ ]:
class BaseEncoder(nn.Module):
    """
    A flexible encoder model that supports different architectures (ResNet, EfficientNet, etc.).
    """

    def __init__(
        self,
        arch="resnet18",
        pretrained=True,
        no_channels=3,
        dropout=0.5,
        add_block=False,
        num_frozen=0,
    ):
        super(BaseEncoder, self).__init__()

        self.add_block = add_block
        self.num_frozen = num_frozen

        # Dictionary of available architectures
        arch_dict = {
            # ResNets
            "resnet18": models.resnet18,
            "resnet34": models.resnet34,
            "resnet50": models.resnet50,
            # "resnet101": models.resnet101,
            # EfficientNets
            "efficientnet_b0": models.efficientnet_b0,
            "efficientnet_b1": models.efficientnet_b1,
            "efficientnet_b2": models.efficientnet_b2,
            # "efficientnet_b3": models.efficientnet_b3,
            # "efficientnet_b4": models.efficientnet_b4,
            # "efficientnet_b5": models.efficientnet_b5,
            # "efficientnet_b6": models.efficientnet_b6,
            # "efficientnet_b7": models.efficientnet_b7,
            # MobileNets
            "mobilenet_v3_small": models.mobilenet_v3_small,
            "mobilenet_v3_large": models.mobilenet_v3_large,
            # Convnexts
            "convnext_tiny": models.convnext_tiny,
            "convnext_small": models.convnext_small,
            "convnext_base": models.convnext_base,
            # "convnext_large": models.convnext_large,
            # Vision Transformer
            "vit_base_16": models.vit_b_16,
            "vit_base_32": models.vit_b_32,
            # "vit_large_16": models.vit_l_16,
            # "vit_large_32": models.vit_l_32,
            # "vit_huge_14": models.vit_h_14,
            # Swin Transformer V2
            "swinv2_tiny": models.swin_v2_t,
            "swinv2_small": models.swin_v2_s,
            "swinv2_base": models.swin_v2_b,
        }

        assert arch in arch_dict, (
            f"Unsupported architecture: {arch}. Choose from {list(arch_dict.keys())}"
        )

        # Load the model
        self.model = arch_dict[arch](weights="DEFAULT" if pretrained else None)

        if "resnet" in arch:
            self.feature_dim = self.model.fc.in_features
            self.model.fc = nn.Identity()
        elif "efficientnet" in arch:
            self.feature_dim = self.model.classifier[1].in_features
            self.model.classifier = nn.Identity()
        elif "mobilenet" in arch:
            self.feature_dim = self.model.classifier[0].in_features
            self.model.classifier = nn.Identity()
        elif "convnext" in arch:
            self.feature_dim = self.model.classifier[2].in_features  # [LayerNorm, Flatten, Linear]
            self.model.classifier = self.model.classifier[:2]
        elif "vit" in arch:
            self.feature_dim = self.model.heads.head.in_features
            self.model.heads.head = nn.Identity()
        elif "swin" in arch:
            self.feature_dim = self.model.head.in_features
            self.model.head = nn.Identity()

        if no_channels != 3:
            if "resnet" in arch:
                self.model.conv1 = nn.Conv2d(
                    no_channels,
                    self.model.conv1.out_channels,
                    kernel_size=self.model.conv1.kernel_size,
                    stride=self.model.conv1.stride,
                    padding=self.model.conv1.padding,
                    bias=False,
                )

            elif "efficientnet" in arch:
                self.model.features[0][0] = nn.Conv2d(
                    no_channels,
                    self.model.features[0][0].out_channels,
                    kernel_size=self.model.features[0][0].kernel_size,
                    stride=self.model.features[0][0].stride,
                    padding=self.model.features[0][0].padding,
                    bias=False,
                )

            elif "mobilenet" in arch:
                self.model.features[0][0] = nn.Conv2d(
                    no_channels,
                    self.model.features[0][0].out_channels,
                    kernel_size=self.model.features[0][0].kernel_size,
                    stride=self.model.features[0][0].stride,
                    padding=self.model.features[0][0].padding,
                    bias=False,
                )

            elif "convnext" in arch:
                self.model.features[0][0] = nn.Conv2d(
                    no_channels,
                    self.model.features[0][0].out_channels,
                    kernel_size=self.model.features[0][0].kernel_size,
                    stride=self.model.features[0][0].stride,
                    padding=self.model.features[0][0].padding,
                    bias=self.model.features[0][0].bias is not None,
                )

            elif "vit" in arch:  
                self.model.conv_proj = nn.Conv2d(
                    no_channels,
                    self.model.conv_proj.out_channels,
                    kernel_size=self.model.conv_proj.kernel_size,
                    stride=self.model.conv_proj.stride,
                    padding=self.model.conv_proj.padding,
                    bias=self.model.conv_proj.bias is not None,
                )

            elif "swin" in arch:
                first_conv_layer = self.model.features[0][0]  # Access the first Conv2d layer
                self.model.features[0][0] = nn.Conv2d(
                    no_channels, 
                    first_conv_layer.out_channels, 
                    kernel_size=first_conv_layer.kernel_size, 
                    stride=first_conv_layer.stride, 
                    padding=first_conv_layer.padding, 
                    bias=first_conv_layer.bias is not None
                )

        if self.add_block:
            self.addition_block = nn.Sequential(
                nn.Linear(self.feature_dim, self.feature_dim),
                nn.BatchNorm1d(self.feature_dim),
                nn.Dropout(dropout),
                nn.Linear(self.feature_dim, self.feature_dim),
            )

        # self.final_layers = nn.Sequential(
        #     nn.Linear(self.feature_dim, 512),
        #     nn.BatchNorm1d(512),
        #     nn.Dropout(dropout),
        #     nn.Linear(512, 512),
        # )

        self.freeze_layers()

    def freeze_layers(self):
        """
        Freeze the first `num_frozen` layers of the model.
        """
        layers = list(self.model.children())
        assert 0 <= self.num_frozen <= len(layers), (
            f"num_frozen should be between 0 and {len(layers)}"
        )

        for i, child in enumerate(layers):
            if i < self.num_frozen:
                for param in child.parameters():
                    param.requires_grad = False

        print(f"Number of frozen layers: {self.num_frozen}")

    def forward(self, x):
        x = self.model(x)
        if self.add_block:
            x = self.addition_block(x)
        # x = self.final_layers(x)
        return x


In [ ]:
class SigmoidGatedFusion2(nn.Module):
    """Independent Sigmoid-based gating for each input and then concatenates."""
    def __init__(self, in_dim1, in_dim2):
        super(SigmoidGatedFusion, self).__init__()
        self.attn1 = nn.Linear(in_dim1, in_dim1)  # Produces weighted features
        self.attn2 = nn.Linear(in_dim2, in_dim2)

    def forward(self, x1, x2):
        weight1 = torch.sigmoid(self.attn1(x1))  
        weight2 = torch.sigmoid(self.attn2(x2))  
        fused = torch.cat((weight1 * x1, weight2 * x2), dim=1)  # Concatenate weighted features
        return fused

class SigmoidGatedFusion(nn.Module):
    """Independent Sigmoid-based gating for each input and then concatenates."""
    def __init__(self, in_dim1, in_dim2):
        super(SigmoidGatedFusion, self).__init__()
        self.attn1 = nn.Linear(in_dim1, 1)  # Produces weighted features
        self.attn2 = nn.Linear(in_dim2, 1)

    def forward(self, x1, x2):
        weight1 = torch.sigmoid(self.attn1(x1))  
        weight2 = torch.sigmoid(self.attn2(x2))  
        fused = torch.cat((weight1 * x1, weight2 * x2), dim=1)  # Concatenate weighted features
        return fused


class SoftmaxGatedFusion(nn.Module):
    """Softmax-based gating where attention weights sum to 1."""
    def __init__(self, in_dim1, in_dim2):
        super(SoftmaxGatedFusion, self).__init__()
        self.gate = nn.Linear(in_dim1 + in_dim2, 2)  # Two attention scores

    def forward(self, x1, x2):
        combined_features = torch.cat((x1, x2), dim=1)  
        weights = torch.softmax(self.gate(combined_features), dim=1)  # Softmax over two inputs
        weighted_x1 = weights[:, 0:1] * x1
        weighted_x2 = weights[:, 1:2] * x2
        fused = torch.cat((weighted_x1, weighted_x2), dim=1)  # Concatenate weighted features
        return fused

# class CrossAttention(nn.Module):
#     """Multihead Cross Attention treating one input as query and other as key/value, ."""
#     def __init__(self, in_dim1, in_dim2, num_heads=4):
#         super(CrossAttention, self).__init__()
#         self.query_proj = nn.Linear(in_dim1, in_dim1)
#         self.key_proj = nn.Linear(in_dim2, in_dim1)
#         self.value_proj = nn.Linear(in_dim2, in_dim1)
#         self.attn = nn.MultiheadAttention(embed_dim=in_dim1, num_heads=num_heads, batch_first=True)

#     def forward(self, x1, x2):
#         q = self.query_proj(x1).unsqueeze(1)  # Query from x1
#         k = self.key_proj(x2).unsqueeze(1)    # Key from x2
#         v = self.value_proj(x2).unsqueeze(1)  # Value from x2
#         attn_output, _ = self.attn(q, k, v)  
#         attn_output = attn_output.squeeze(1)
#         fused = torch.cat((attn_output, x1, x2), dim=1)  # Concatenate attention output with original features
#         return fused

class CrossAttention(nn.Module):
    """Multihead Cross Attention treating one input as query and other as key/value."""
    def __init__(self, in_dim1, in_dim2, shared_dim=256, num_heads=4):
        super(CrossAttention, self).__init__()
        
        # Project both inputs to a shared dimension
        self.query_proj = nn.Linear(in_dim1, shared_dim)
        self.key_proj = nn.Linear(in_dim2, shared_dim)
        self.value_proj = nn.Linear(in_dim2, shared_dim)

        # Multihead attention with consistent dimensions
        self.attn = nn.MultiheadAttention(embed_dim=shared_dim, num_heads=num_heads, batch_first=True)

    def forward(self, x1, x2):
        # Project to shared dimension
        q = self.query_proj(x1).unsqueeze(1)  # (batch_size, 1, shared_dim)
        k = self.key_proj(x2).unsqueeze(1)    # (batch_size, 1, shared_dim)
        v = self.value_proj(x2).unsqueeze(1)  # (batch_size, 1, shared_dim)
    
        # Attention
        attn_output, _ = self.attn(q, k, v)
        attn_output = attn_output.squeeze(1)
    
        # Ensure consistent concatenation
        fused = torch.cat((attn_output, x1, x2), dim=1)
    
        # 🚀 Add the projection layer here
        projection = nn.Linear(fused.shape[1], 1024).to(fused.device)
        fused = projection(fused)
    
        return fused



def get_attention_module(attention_type, in_dim1, in_dim2, num_heads=4):
    """
    Factory method to return different attention mechanisms for concatenated features.

    Args:
    - attention_type (str): Type of attention ("sigmoid_gated", "softmax_gated", "cross")
    - in_dim1 (int): Feature dimension of input 1
    - in_dim2 (int): Feature dimension of input 2
    - num_heads (int): Number of heads (only for cross attention)

    Returns:
    - nn.Module: The selected attention mechanism
    """
    if attention_type == "sigmoid_gated":
        return SigmoidGatedFusion(in_dim1, in_dim2)
    elif attention_type == "softmax_gated":
        return SoftmaxGatedFusion(in_dim1, in_dim2)
    elif attention_type == "cross":
        return CrossAttention(in_dim1, in_dim2, num_heads)
    else:
        raise ValueError(f"Unknown attention type: {attention_type}")

# AQI model
1. Concats output from 2 models: 1 for satellite features, 1 for street image features
2. Adds final layers (depending on classification, regression)
3. Forming a single model

In [ ]:
class AQIPrediction(nn.Module):
    """
    Unified model
    """
    
    def __init__(self, satellite_model, street_model, dropout=0.5, num_classes=None):
        
        super(AQIPrediction, self).__init__()
        
        self.satellite_model = satellite_model
        self.street_model = street_model
        self.num_classes = num_classes
        
        self.feature_dim = satellite_model.feature_dim + street_model.feature_dim
        
        self.attention = get_attention_module(ATTENTION, satellite_model.feature_dim, street_model.feature_dim)
        
        self.final_layers = nn.Sequential(
            nn.Linear(self.feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        if num_classes:
            self.final_layers[-1] = nn.Linear(128, num_classes)
        else:
            self.aqi_head = nn.Sequential(nn.Linear(128, 1), nn.ReLU())  
            self.pm_head = nn.Sequential(nn.Linear(128, 2), nn.ReLU())   # PM2.5 & PM10
            self.gas_head = nn.Sequential(nn.Linear(128, 4), nn.ReLU())  # O3, CO, SO2, NO2
  
            
            
    def forward(self, street_img, satellite_img):
        
        street_features = self.street_model(street_img)
        satellite_features = self.satellite_model(satellite_img)
        # print(street_features.shape, satellite_features.shape)
        
        # features = torch.cat((street_features, satellite_features), dim=1)
        features = self.attention(satellite_features, street_features)
        
        output = self.final_layers(features)
        if self.num_classes:
            return output
            
        return self.aqi_head(output), self.pm_head(output), self.gas_head(output)
        

# Putting it all together
Creating the training pipeline

In [ ]:
torch.cuda.empty_cache()

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

satellite_encoder = BaseEncoder(arch=SATELLITE_ENCODER, no_channels=7, dropout=DROPOUT, add_block=SATELLITE_EXTRA_LAYER, num_frozen=NUM_FROZEN_LAYERS)
street_encoder = BaseEncoder(arch=STREET_ENCODER, no_channels=3, dropout=DROPOUT, add_block=STREET_EXTRA_LAYER, num_frozen=NUM_FROZEN_LAYERS)

model = AQIPrediction(satellite_encoder, street_encoder, dropout=DROPOUT, num_classes=None)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS)

history = train(model, optimizer, scheduler, train_loader, val_loader, test_loader, device, EPOCHS, f'{RUN_NAME}_best_model.pth')

plot_metrics(**history)

In [ ]:
run.finish()